In [2]:
import nltk
import argparse
import librosa #audio lib
import torch
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

access_token = 'hf_hf_jBwrxIyVtywIqBARmBenwvdJKvBhqThLYb'

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ix502iv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def load_wav2vec_960h_model():
  """
  returns the tokenizer and the model from pretrained tokenizer models
  """
  tokenizer = Wav2Vec2Tokenizer.from_pretrained('facebook/wav2vec-base-960h',
                                                use_auth_token=access_token)
  model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-base-960h',
                                         use_auth_token=access_token)
  return tokenizer, model

def correct_uppercase_sentence(input_text):
  """
  returns the correct sentence
  """
  sentences = nltk.sent_tokenize(input_text)
  return(' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))

In [4]:
def asr_transcription(tokenizer, model, input_file):
  """
  Returns the transcript of the input auio recording
  output: transcribed text
  input: hugglingface tokenizer, model, wav file
  """

  #read the file
  speech, samplerate = sf.read(input_file)
  #make it 1-dimentional
  if len(speech.shape)>1:
    speech = speech[:,0] + speech[:,1]
  #resample to 16kHz
  if samplerate != 16000:
    speech = librosa.resample(speech, samplerate, 16000)
  #tokenize
  input_values = tokenizer(speech, return_tensors='pt').input_values
  #take logits
  logits = model(input_values).logits
  #take argmax( find most probable word id)
  predicted_ids = torch.argmax(logits, dim=-1)
  #get the word from the predicted words ids
  transcription = tokenizer.decode(predicted_ids[0])
  #output all in uppercase
  transcription = correct_uppercase_sentence(transcription.lower())
  return transcription

In [5]:
wav_input = "jamoh_say_sth.wav"
tokenizer, model = load_wav2vec_960h_model()
text = asr_transcription(tokenizer, model, wav_input)
print(text)

OSError: facebook/wav2vec-base-960h is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.